# Import libraries and models

In [0]:
import ast
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

import keras
import tensorflow
from tensorflow.keras.callbacks import EarlyStopping 
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Activation, Flatten, BatchNormalization
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Conv1D, MaxPooling1D
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.models import model_from_json
from keras.preprocessing.text import Tokenizer
from keras.layers import Embedding

from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score
from sklearn.linear_model import LogisticRegression, SGDClassifier
from sklearn import metrics

import re
import operator

from tqdm.auto import tqdm
tqdm.pandas()
from joblib import dump, load


Using TensorFlow backend.


In [0]:
# # Import v1 logreg
# logreg_v1 = load('/content/drive/My Drive/MScA/Machine Learning/ML Final Project/Models/game_cat_logreg.joblib') 

# # Import v1 CNN
# json_file = open('/content/drive/My Drive/MScA/Machine Learning/ML Final Project/Models/dl_gameFeature_model_base.json', 'r')
# loaded_model_json = json_file.read()
# json_file.close()
# cnn_v1 = model_from_json(loaded_model_json)
# cnn_v1.load_weights("/content/drive/My Drive/MScA/Machine Learning/ML Final Project/Models/dl_gameFeature_model_base.h5")
# cnn_v1.compile(loss=keras.losses.categorical_crossentropy,
#               optimizer=tensorflow.keras.optimizers.Adam(),
#               metrics=['accuracy'])
# print("Loaded model from disk")

# # Import v2 logreg
# logreg_v2 = load('/content/drive/My Drive/MScA/Machine Learning/ML Final Project/Models/game_cat_logreg_v2.joblib') 

# # Import v2 CNN
# json_file = open('/content/drive/My Drive/MScA/Machine Learning/ML Final Project/Models/dl_gameFeature_model_base_adam_v2.json', 'r')
# loaded_model_json = json_file.read()
# json_file.close()
# cnn_v2 = model_from_json(loaded_model_json)
# cnn_v2.load_weights("/content/drive/My Drive/MScA/Machine Learning/ML Final Project/Models/dl_gameFeature_model_base_adam_v2.h5")
# cnn_v2.compile(loss=keras.losses.categorical_crossentropy,
#               optimizer=tensorflow.keras.optimizers.Adam(),
#               metrics=['accuracy'])
# print("Loaded model from disk")


# Import v3 logreg
logreg_v3 = load('/content/drive/My Drive/MScA/Machine Learning/ML Final Project/Models/description_logreg_v1.joblib') 

# Import v3 nb
nb_v3 = load('/content/drive/My Drive/MScA/Machine Learning/ML Final Project/Models/description_naiveBayes_v1.joblib') 

# Import RNN
json_file = open('/content/drive/My Drive/MScA/Machine Learning/ML Final Project/Models/gru_bidir.json', 'r')
loaded_model_json = json_file.read()
json_file.close()
rnn_v1 = model_from_json(loaded_model_json)
rnn_v1.load_weights("/content/drive/My Drive/MScA/Machine Learning/ML Final Project/Models/gru_bidir.h5")
rnn_v1.compile(loss=keras.losses.categorical_crossentropy,
              optimizer=tensorflow.keras.optimizers.Adam(),
              metrics=['accuracy'])
print("Loaded model from disk")

/usr/local/lib/python3.6/dist-packages/sklearn/utils/deprecation.py:144: FutureWarning: The sklearn.linear_model.logistic module is  deprecated in version 0.22 and will be removed in version 0.24. The corresponding classes / functions should instead be imported from sklearn.linear_model. Anything that cannot be imported from sklearn.linear_model is now part of the private API.
  warnings.warn(message, FutureWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/base.py:318: UserWarning: Trying to unpickle estimator LogisticRegression from version 0.21.3 when using version 0.22.2.post1. This might lead to breaking code or invalid results. Use at your own risk.
  UserWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/base.py:318: UserWarning: Trying to unpickle estimator MultinomialNB from version 0.21.3 when using version 0.22.2.post1. This might lead to breaking code or invalid results. Use at your own risk.
  UserWarning)


Loaded model from disk


# Prepare model inputs

## Load full data

In [0]:
#features = pd.read_csv("/content/drive/My Drive/ml_data/features_clean.csv")
features = pd.read_csv("/content/drive/My Drive/MScA/Machine Learning/ML Final Project/Data/descriptions_clean.csv")
features.dropna(subset=["description_joined"],inplace=True)

/usr/local/lib/python3.6/dist-packages/IPython/core/interactiveshell.py:2718: DtypeWarning: Columns (1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


## Load vectorizer and tokenizer

In [0]:
# Vectorizers for feature texts

# vectorizer_v1 = load("/content/drive/My Drive/MScA/Machine Learning/ML Final Project/Models/base_vectorizer_v1.joblib")
# games_feat_vec_v1 = vectorizer_v1.transform(features.features_joined.tolist()) # transform full input

vectorizer_desc = load("/content/drive/My Drive/MScA/Machine Learning/ML Final Project/Models/description_vectorizer_v1.joblib")
games_feat_vec_desc = vectorizer_desc.transform(features.description_joined.tolist()) # transform full input

/usr/local/lib/python3.6/dist-packages/sklearn/base.py:318: UserWarning: Trying to unpickle estimator CountVectorizer from version 0.21.3 when using version 0.22.2.post1. This might lead to breaking code or invalid results. Use at your own risk.
  UserWarning)


In [0]:
# Vectorize for CNN
t = load("/content/drive/My Drive/MScA/Machine Learning/ML Final Project/Models/description_tokenizer.joblib") 
games_docs = np.array(features.description_joined).tolist()
encoded_games_docs = t.texts_to_sequences(games_docs)
padded_games_docs = tensorflow.keras.preprocessing.sequence.pad_sequences(encoded_games_docs, maxlen=500, padding='post')

# Get Outputs from models

## LogReg Output

In [0]:
# LogReg v1
# games_logreg_preds_v1 = logreg_v1.predict(games_feat_vec_v1) # Predictions for LogReg v1
# logreg_preds_proba_v1 = logreg_v1.predict_proba(games_feat_vec_v1)

In [0]:
# LogReg v2
# games_logreg_preds_v2 = logreg_v2.predict(games_feat_vec_v2) # Predictions for LogReg v2
# logreg_preds_proba_v2 = logreg_v2.predict_proba(games_feat_vec_v2)

In [0]:
# LogReg desc

games_logreg_preds_desc = logreg_v3.predict(games_feat_vec_desc) # Predictions for LogReg v3
logreg_preds_proba_desc = logreg_v3.predict_proba(games_feat_vec_desc)

In [0]:
# NB desc

games_nb_preds_desc = nb_v3.predict(games_feat_vec_desc) # Predictions for LogReg v3
nb_preds_proba_desc = nb_v3.predict_proba(games_feat_vec_desc)

## CNN Output

In [0]:
# CNN v1
# games_cnn_preds_v1 = cnn_v1.predict_classes(padded_games_docs)
# cnn_preds_proba_v1 = cnn_v1.predict(padded_games_docs)

Instructions for updating:
Please use instead:* `np.argmax(model.predict(x), axis=-1)`,   if your model does multi-class classification   (e.g. if it uses a `softmax` last-layer activation).* `(model.predict(x) > 0.5).astype("int32")`,   if your model does binary classification   (e.g. if it uses a `sigmoid` last-layer activation).


In [0]:
# # CNN v2
# games_cnn_preds_v2 = cnn_v2.predict_classes(padded_games_docs)
# cnn_preds_proba_v2 = cnn_v2.predict(padded_games_docs)

In [0]:
# RNN v1
rnn_preds_proba_v1 = rnn_v1.predict(padded_games_docs)
games_rnn_preds_v1 = rnn_preds_proba_v1.argmax(axis=-1)


In [0]:
from collections import Counter
Counter(games_rnn_preds_v1)

Counter({0: 335,
         1: 2282,
         2: 157,
         3: 313,
         4: 541,
         5: 198,
         6: 821,
         7: 394,
         8: 1683,
         10: 131,
         11: 253,
         12: 2,
         13: 326,
         14: 1035,
         15: 663,
         16: 405,
         17: 491,
         18: 12378,
         19: 1371,
         20: 2866,
         21: 593})

# Aggregate category probabilities by game
Simple average of considered model's predicted probabilities

In [0]:
logreg_preds_proba_desc.shape

(27238, 22)

In [0]:
nb_preds_proba_desc.shape

(27238, 22)

In [0]:
rnn_preds_proba_v1.shape

(27238, 22)

In [0]:
array1 = logreg_preds_proba_desc[:,:,np.newaxis]
array2 = nb_preds_proba_desc[:,:,np.newaxis]
array3 = rnn_preds_proba_v1[:,:,np.newaxis]
proba_means = np.mean(np.concatenate((array1,array2,array3), axis = 2),axis=2)
proba_df = pd.DataFrame(proba_means)
ids = features.asin.tolist()
proba_df["productID"] = ids
proba_df

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,productID
0,0.000927,0.663799,0.000039,0.000029,0.000464,0.000555,0.001492,0.000912,0.000741,0.000051,0.000046,0.000274,0.000322,0.000042,0.001735,0.000717,0.000267,0.000449,0.322348,0.001474,0.002997,0.000320,0439335310
1,0.017509,0.078878,0.005404,0.019977,0.028995,0.016772,0.047489,0.029170,0.038341,0.004848,0.018106,0.018594,0.010457,0.041109,0.030652,0.024409,0.038886,0.042646,0.352370,0.029583,0.060733,0.045069,0439339006
2,0.012526,0.070359,0.011391,0.019072,0.018790,0.023021,0.027187,0.031538,0.045587,0.003711,0.015177,0.018577,0.013878,0.023528,0.018696,0.025422,0.027644,0.037165,0.386852,0.063006,0.071435,0.035437,043933702X
3,0.000302,0.000969,0.000311,0.000066,0.003829,0.000117,0.001396,0.000144,0.189689,0.000045,0.003567,0.000158,0.000266,0.000066,0.022651,0.000942,0.005781,0.000579,0.029984,0.057175,0.681795,0.000169,0439339987
4,0.007385,0.032591,0.003390,0.076898,0.010478,0.007003,0.012821,0.008481,0.023183,0.002279,0.004014,0.011004,0.012215,0.010681,0.205564,0.017459,0.012371,0.022678,0.353851,0.105753,0.049953,0.009948,0439342260
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
27233,0.002823,0.024435,0.000856,0.001784,0.020914,0.004905,0.021954,0.004953,0.022724,0.000760,0.001801,0.006555,0.003897,0.002276,0.007953,0.014628,0.002502,0.003093,0.663982,0.141306,0.040584,0.005314,B01HI8JR82
27234,0.000028,0.000502,0.000024,0.000022,0.000010,0.000165,0.000827,0.000410,0.002180,0.000005,0.000018,0.000035,0.000057,0.000029,0.007135,0.000048,0.000947,0.000910,0.838712,0.001706,0.146192,0.000039,B01HIZF83S
27235,0.001858,0.003606,0.001327,0.001273,0.004730,0.001771,0.006998,0.000469,0.003753,0.000169,0.001397,0.003484,0.000894,0.006584,0.015598,0.046290,0.025824,0.038286,0.792713,0.011949,0.026726,0.004302,B01HIZGKOE
27236,0.000028,0.000502,0.000024,0.000022,0.000010,0.000165,0.000827,0.000410,0.002180,0.000005,0.000018,0.000035,0.000057,0.000029,0.007135,0.000048,0.000947,0.000910,0.838712,0.001706,0.146192,0.000039,B01HIZF7XE


In [0]:
dump(proba_df,"/content/drive/My Drive/MScA/Machine Learning/ML Final Project/Data/game_categories_probabilities.joblib")

['/content/drive/My Drive/MScA/Machine Learning/ML Final Project/Data/game_categories_probabilities.joblib']

# Combine models
Old version of ensemble where we considered the biggest probability

In [0]:
# Column 1 = asin
# Column 2 = feat_text
# Column 3 = logreg_preds_proba
# Column 4 = games_logreg_preds
# Column 5 = cnn_preds_proba
# Column 6 = games_cnn_preds

asin = features.asin
feat_text = features.features_joined
game_feat_preds = pd.concat([asin,feat_text], axis = 1)
# game_feat_preds['logreg_preds_proba_v1'] = np.max(logreg_preds_proba_v1, axis=-1)
# game_feat_preds['games_logreg_preds_v1'] = games_logreg_preds_v1
game_feat_preds['logreg_preds_proba_v2'] = np.max(logreg_preds_proba_v2, axis=-1)
game_feat_preds['games_logreg_preds_v2'] = games_logreg_preds_v2
# game_feat_preds['cnn_preds_proba_v1'] = np.max(cnn_preds_proba_v1, axis=-1)
# game_feat_preds['games_cnn_preds_v1'] = games_cnn_preds_v1
game_feat_preds['cnn_preds_proba_v2'] = np.max(cnn_preds_proba_v2, axis=-1)
game_feat_preds['games_cnn_preds_v2'] = games_cnn_preds_v2

In [0]:
# Using all models
full_preds = []
for i,r in game_feat_preds.iterrows():
  list_probas = [game_feat_preds.loc[i,'cnn_preds_proba_v1'],
                 game_feat_preds.loc[i,'cnn_preds_proba_v2'],
                 game_feat_preds.loc[i,'logreg_preds_proba_v1'],
                 game_feat_preds.loc[i,'logreg_preds_proba_v2']]
  list_preds = [game_feat_preds.loc[i,'games_cnn_preds_v1'],
                game_feat_preds.loc[i,'games_cnn_preds_v2'],
                game_feat_preds.loc[i,'games_logreg_preds_v1'],
                game_feat_preds.loc[i,'games_logreg_preds_v2']]
  best_pred = list_preds[np.argmax(list_probas)] 
  full_preds.append(best_pred)


# Using new models
new_preds = []
for i,r in game_feat_preds.iterrows():
  list_probas = [game_feat_preds.loc[i,'cnn_preds_proba_v2'],
                 game_feat_preds.loc[i,'logreg_preds_proba_v2']]
  list_preds = [game_feat_preds.loc[i,'games_cnn_preds_v2'],
                game_feat_preds.loc[i,'games_logreg_preds_v2']]
  best_pred = list_preds[np.argmax(list_probas)] 
  new_preds.append(best_pred)

In [0]:
game_feat_preds["full_preds"] = full_preds
game_feat_preds["new_preds"] = new_preds
game_feat_preds["game"] = features['title']
game_feat_preds.head(5)

,asin,features_joined,logreg_preds_proba_v1,games_logreg_preds_v1,logreg_preds_proba_v2,games_logreg_preds_v2,cnn_preds_proba_v1,games_cnn_preds_v1,cnn_preds_proba_v2,games_cnn_preds_v2,full_preds,new_preds,game
0,0439335310,Grades 2-12 Spelling Program Teaches Spelling ...,0.088128,26,0.257119,25,0.814444,15,0.158219,7,15,25,Phonics Alive! 3: The Speller
1,0439339006,Sim City 3000 CD-ROM,0.127250,10,0.401816,25,0.178567,9,0.358299,25,25,25,Sim City 3000
2,0439335299,Phonics Alive! 2 The Sound Blender is a new an...,0.915462,12,0.921971,12,0.313014,12,0.199003,12,12,12,Phonics Alive! 2 The Sound Blender
3,043933702X,video game,0.150539,27,0.435079,25,0.190575,14,0.343529,26,25,25,Need for Speed Porsche Unleashed
4,0439339987,"Like the real world, Freddi Fishs world change...",0.936875,7,0.945029,7,0.267910,15,0.235829,26,7,7,Freddi Fish and the Case of The Missing Kelp S...


In [0]:
# Create feature df
feat_val_df = pd.DataFrame({
'platformer' :0 
,'shooter' :1 
,'arcade' :3 
,'fighting' :5 
,'stealth' :7 
,'survival' :8 
,'horror' :9 
,'rhythm' : 10
,'metroidvania' : 11
,'action_adventure' : 12
,'visual_novels' : 14
,'rpg_action' : 15
,'rpg_mmo' : 16
,'rpg_roguelike' : 17
,'rpg_tactical' : 18
,'rpg_sandbox' : 19
,'sim_management' : 20
,'sim_vehicle' : 21
,'strat' : 22
,'sports' : 25
,'puzzle' : 26
,'casual_party' : 27
,'casual_family' : 28},index=[0]).T
feat_val_df.reset_index(inplace = True)
feat_val_df.columns = ["cat", "val"]
feat_val_df

,cat,val
0,platformer,0
1,shooter,1
2,arcade,3
3,fighting,5
4,stealth,7
5,survival,8
6,horror,9
7,rhythm,10
8,metroidvania,11
9,action_adventure,12


In [0]:
# Translate class values
game_feat_preds = game_feat_preds.merge(feat_val_df, how = "left", left_on = "new_preds", right_on = "val")
game_feat_preds.drop("val", axis = 1, inplace = True)

In [0]:
# Pick random rows to check
game_feat_preds.sample(n=10).loc[:,"game":]

,game,cat_x,cat_y
15929,The World's Best: Card Games (JC) - PC,action_adventure,action_adventure
22769,IGT Slots: Little Green Men - PC/Mac,rpg_action,sports
3396,Game Controller for Para Para Paradise,casual_party,casual_party
27454,Snack 2 Go - Match 3 (in German) [Download],fighting,fighting
16968,Jak & Daxter: The Lost Frontier - PlayStation 2,survival,survival
19602,ATV Mudracer [Download],rpg_mmo,sports
6218,Instant Landscaping,horror,horror
8615,Zoo Tycoon DS,platformer,sports
7009,American Chopper - PC,sim_vehicle,sim_vehicle
16260,World Airports 3 North America Expansion for F...,sports,sports


In [0]:
game_feat_preds = game_feat_preds.rename(columns={'cat_x': "full_model_prediction",'cat_y':"new_model_predictions"})
dump(game_feat_preds,"/content/drive/My Drive/MScA/Machine Learning/ML Final Project/Data/game_feature_predictions.joblib")

['/content/drive/My Drive/MScA/Machine Learning/ML Final Project/Data/game_feature_predictions.joblib']

In [0]:
game_feat_preds = load("/content/drive/My Drive/MScA/Machine Learning/ML Final Project/Data/game_feature_predictions.joblib")

In [0]:
game_feat_preds.sample(n=10).loc[:,"game":]

,game,full_model_prediction,new_model_predictions
14790,var aPageStart = (new Date()).getTime();\nvar ...,survival,sports
27855,NBA 2K17 - Early Tip Off Edition - PlayStation 4,casual_party,rhythm
12498,Soul Nomad & The World Eaters,rpg_mmo,rpg_tactical
17074,Petz Dogz Family - Sony PSP,puzzle,puzzle
20263,Resident Evil: Operation Raccoon City Special ...,casual_party,sports
16548,Mystery Chronicles - Murder Among Friends,casual_party,casual_party
16087,var aPageStart = (new Date()).getTime();\nvar ...,sports,sports
25881,Phantom Breaker Battle Grounds [Online Game Code],arcade,arcade
9076,Gateway II: Homeworld,sim_vehicle,sim_vehicle
6131,Mafia (Jewel Case) - PC,puzzle,puzzle
